# json -> pd.DataFrame -> arrow

In [2]:
import json
import pandas as pd
from typing import List
import pyarrow as pa
import pyarrow.parquet as pq

## 1) json -> pd.DataFrame

원본 파일에서 "documents" 정보만 빼와서 새로운 파일로 저장

In [1]:
with open("/opt/ml/dataset/Training/train_original.json", "r") as f:  # ai hub 신문기사 json 파일
    data = json.load(f)
    data = data["documents"]

with open("/opt/ml/dataset/Training/train.json", "w") as f:  # 새로 저장
    json.dump(data, f, indent=4, ensure_ascii=False)  # 한글이라 ensure_ascii=False

In [4]:
news_df = pd.read_json("/opt/ml/dataset/Training/train.json")
news_df.head()

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,290741794,정치,online,지역지,광양신문,small,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,290741797,종합,online,지역지,광양신문,small,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...


데이터 기본 정보 확인

In [25]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271093 entries, 0 to 271092
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   id                       271093 non-null  int64 
 1   category                 271093 non-null  object
 2   media_type               271093 non-null  object
 3   media_sub_type           271093 non-null  object
 4   media_name               271093 non-null  object
 5   size                     271093 non-null  object
 6   char_count               271093 non-null  int64 
 7   publish_date             271093 non-null  object
 8   title                    271093 non-null  object
 9   text                     271093 non-null  object
 10  annotator_id             271093 non-null  int64 
 11  document_quality_scores  271093 non-null  object
 12  extractive               271093 non-null  object
 13  abstractive              271093 non-null  object
dtypes: int64(3), object(

In [26]:
news_df["id"].nunique  # 271093
# news_df["title"].nunique  # 271093
# news_df["text"].nunique  # 271093
# news_df["extractive"].unique  # 271093 - ㅇㄱㄹㅇ?
# news_df["abstractive"].nunique  # 271093

<bound method IndexOpsMixin.nunique of 0         290741778
1         290741792
2         290741793
3         290741794
4         290741797
            ...    
271088    340626140
271089    340626209
271090    340626213
271091    340626365
271092    340626537
Name: id, Length: 271093, dtype: int64>

In [71]:
# 필요한 정보만
news_df_final = news_df[['id', 'char_count', 'title', 'text', 'extractive', 'abstractive']]
news_df_final.head()

,id,char_count,title,text,extractive,abstractive
0,290741778,927,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,764,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,1066,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,290741794,746,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,290741797,978,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...


## text 재구성

text 예시

In [60]:
news_df_final["text"][0]  # List[List[Dict]]

[[{'index': 0,
   'sentence': 'ha당 조사료 400만원…작물별 차등 지원',
   'highlight_indices': ''}],
 [{'index': 1, 'sentence': '이성훈 sinawi@hanmail.net', 'highlight_indices': ''}],
 [{'index': 2,
   'sentence': '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
   'highlight_indices': ''}],
 [{'index': 3,
   'sentence': '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
   'highlight_indices': '35,37'}],
 [{'index': 4,
   'sentence': '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
   'highlight_indices': '9,11;33,34'},
  {'index': 5,
   'sentence': '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
   'highlight_indices': '50,52'}],
 [{'index': 6,
   'sentence': '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
   'highlight_indices': '24,26'}],
 [{'index': 7,
   'sentence': '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
   'highlight_ind

이유는 모르겠지만 중간에 빈 리스트도 껴있다.

In [62]:
news_df_final["text"][1]

[[{'index': 0,
   'sentence': '8억 투입, 고소천사벽화·자산마을에 색채 입혀',
   'highlight_indices': ''}],
 [{'index': 1, 'sentence': '이성훈 sinawi@hanmail.net', 'highlight_indices': ''}],
 [],
 [{'index': 2,
   'sentence': '여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의 도시를 완성했다.',
   'highlight_indices': ''},
  {'index': 3,
   'sentence': '시에 따르면 총 사업비 8억원이 투입된 컬러빌리지 사업은‘낮에는 색채, 밤에는 빛’을 주제로 지난해 초 착공에 들어갔다.',
   'highlight_indices': '7,8'}],
 [{'index': 4,
   'sentence': '컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었다.',
   'highlight_indices': '49,53'}],
 [{'index': 5,
   'sentence': '특히 자산마을 주택 70여 동은 동백꽃 색으로 새 단장을 마쳤다.',
   'highlight_indices': '0,2;26,27'},
  {'index': 6,
   'sentence': '마을 옆을 지나는 해상케이블카에서 내려다보면 동백꽃 한 송이가 피어있는 모습으로 SNS 등을 통해 입소문을 타고 있다.',
   'highlight_indices': '29,30'}],
 [{'index': 7,
   'sentence': '고소천사벽화마을 주택 130여 동도 지붕은 동백꽃 색, 벽면은 파스텔 톤 색상으로 변신해 이국적인 분위기를 내고 있다.',
   'highlight_indices': ''}],
 [{'index': 8,
   'sentence': '마을 내 옹벽과 비탈면은 패턴벽화와 야간조명이 설치돼 포토존으로 

In [63]:
def to_list_str(text) -> List[str]:
    result = []
    for item in text:
        if len(item) < 1: continue
        result.append(item[0]["sentence"])
    return result
    
# 예시
to_list_str(news_df_final["text"][1])

['8억 투입, 고소천사벽화·자산마을에 색채 입혀',
 '이성훈 sinawi@hanmail.net',
 '여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의 도시를 완성했다.',
 '컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었다.',
 '특히 자산마을 주택 70여 동은 동백꽃 색으로 새 단장을 마쳤다.',
 '고소천사벽화마을 주택 130여 동도 지붕은 동백꽃 색, 벽면은 파스텔 톤 색상으로 변신해 이국적인 분위기를 내고 있다.',
 '마을 내 옹벽과 비탈면은 패턴벽화와 야간조명이 설치돼 포토존으로 인기를 끌고 있다.',
 '시는 사업대상이 사유시설인 만큼 계획단계부터 주민들의 의견을 들었다.',
 '시 관계자는“컬러빌리지 사업은 주민들의 적극적인 관심과 참여가 만들어낸 값진 성과물”이라며“돌산공원, 해상케이블카, 남산공원, 돌산대교 등에서 빛과 색채로 물들어가는 여수의 낭만을 만끽하시기 바란다”고 말했다.']

In [65]:
news_df_final["text"] = news_df_final["text"].apply(lambda x: to_list_str(x))
news_df_final.head()

/tmp/ipykernel_52377/2113709001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df_final["text"] = news_df_final["text"].apply(lambda x: to_list_str(x))


,id,char_count,title,text,extractive,abstractive
0,290741778,927,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,764,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,1066,“새해 정기 받고 올해는 반드시 일내자!”,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...
3,290741794,746,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화..."
4,290741797,978,타이완 크루즈관광객 4천여명‘전남’온다,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...","[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...


결과 확인

In [68]:
news_df_final["text"][3]

['11~24일, 매실·감·참다래 등 지역특화작목',
 '이성훈 sinawi@hanmail.net',
 '광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050명을 대상으로 새해 농업인 실용교육을 실시한다.',
 '이번 교육은 농산물 FTA 확대와 경기침체 등 국내외 농업여건의 어려워짐에 따라 농업인들의 경쟁력을 높이고, 소득안정을 위해 마련됐다.',
 '교육은 시의 특화작목인 감, 참다래, 생강, 매실 등에 대한 저비용 생산과 고품질 재배기술을 중점적으로 다룬다.',
 '시는 그동안 주요소득작목을 중심으로 연중 체계적인 교육과 현장학습을 통해 농업인들의 전문기술 향상을 이끌어 내고 있다.',
 '이정헌 기술지원팀장은“이번 교육은 국립특작과학원과 전남농업기술원 등에서 전문가를 초빙해 품목별로 내실 있는 교육으로 진행된다”며,“많은 농가에서 이번 교육에 참여해 유익한 농업기술정보를 가져가는 시간이 되었으면 한다”고 말했다.',
 '자세한 교육 일정은 광양시 농업기술센터 홈페이지 농업인교육에서 확인할 수 있으며, 교육을 희망하는 농업인은 기술보급과 기술지원팀(797-3323)이나 읍면동 농업인상담소로 문의하면 된다.']

## 2) pd.DataFrame -> arrow

In [ ]:
# import pyarrow as pa
# import pyarrow.parquet as pq

In [69]:
news_table = pa.Table.from_pandas(news_df_final)

In [ ]:
news_table

"aihub_news.parquet" 로 저장

In [73]:
pq.write_table(news_table, '/opt/ml/dataset/Training/train.parquet')  # preserve_index=False (omit index)

파일 읽어오기

In [74]:
reloaded_news_table = pq.read_table('/opt/ml/dataset/Training/train.parquet')

In [75]:
type(reloaded_news_table)  # pyarrow.lib.Table

pyarrow.lib.Table

# validation set도 동일하게 진행

In [3]:
with open("/opt/ml/dataset/Validation/valid_original.json", "r") as f:
    data = json.load(f)
    data = data["documents"]

with open("/opt/ml/dataset/Validation/valid.json", "w") as f:  # 새로 저장
    json.dump(data, f, indent=4, ensure_ascii=False)  # 한글이라 ensure_ascii=False

In [4]:
val_news_df = pd.read_json("/opt/ml/dataset/Validation/valid.json")
val_news_df.head()

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,340626877,정치,online,경제지,한국경제,medium,1111,2019-04-08 17:43:00,"文대통령 ""5G는 4차산업혁명 시대의 고속도로""","[[{'index': 0, 'sentence': '[ 박재원 기자 ] '대한민국 5...",4209,"{'readable': 5, 'accurate': 5, 'informative': ...","[0, 1, 3]",[8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 ...
1,340626896,종합,online,경제지,한국경제,small,904,2019-04-08 17:41:00,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'","[[{'index': 0, 'sentence': '] 당 지도부 퇴진을 놓고 바른미...",2438,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 1, 6]",[8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 ...
2,340626904,"IT,과학",online,경제지,한국경제,medium,1104,2019-04-08 17:44:00,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연","[[{'index': 0, 'sentence': '[ 홍윤정 기자 ] 8일 서울 올...",178,"{'readable': 5, 'accurate': 5, 'informative': ...","[1, 5, 8]",[지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리...
3,340627450,사회,online,경제지,한국경제,small,740,2019-04-08 17:48:00,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순","[[{'index': 0, 'sentence': '] 박원순 서울시장(사진)이 8일...",1629,"{'readable': 3, 'accurate': 3, 'informative': ...","[0, 1, 2]",[박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하...
4,340627465,경제,online,경제지,한국경제,medium,1389,2019-04-08 17:49:00,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한","[[{'index': 0, 'sentence': '[ 임근호 기자 ] ""SK(주)와...",2411,"{'readable': 4, 'accurate': 3, 'informative': ...","[1, 3, 4]",[주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적...


In [5]:
val_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30122 entries, 0 to 30121
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       30122 non-null  int64 
 1   category                 30122 non-null  object
 2   media_type               30122 non-null  object
 3   media_sub_type           30122 non-null  object
 4   media_name               30122 non-null  object
 5   size                     30122 non-null  object
 6   char_count               30122 non-null  int64 
 7   publish_date             30122 non-null  object
 8   title                    30122 non-null  object
 9   text                     30122 non-null  object
 10  annotator_id             30122 non-null  int64 
 11  document_quality_scores  30122 non-null  object
 12  extractive               30122 non-null  object
 13  abstractive              30122 non-null  object
dtypes: int64(3), object(11)
memory usage: 

In [12]:
val_news_df["id"].nunique  # 30122
# val_news_df["title"].nunique  # 30122
# val_news_df["text"].nunique  # 30122
# val_news_df["extractive"].unique  # 30122 - ㅇㄱㄹㅇ?
# val_news_df["abstractive"].nunique  # 30122

<bound method IndexOpsMixin.nunique of 0        340626877
1        340626896
2        340626904
3        340627450
4        340627465
           ...    
30117    350851474
30118    350851925
30119    350854748
30120    350857648
30121    350861693
Name: id, Length: 30122, dtype: int64>

In [13]:
# 필요한 정보만
val_news_df_final = val_news_df[['id', 'char_count', 'title', 'text', 'extractive', 'abstractive']]
val_news_df_final.head()

,id,char_count,title,text,extractive,abstractive
0,340626877,1111,"文대통령 ""5G는 4차산업혁명 시대의 고속도로""","[[{'index': 0, 'sentence': '[ 박재원 기자 ] '대한민국 5...","[0, 1, 3]",[8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 ...
1,340626896,904,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'","[[{'index': 0, 'sentence': '] 당 지도부 퇴진을 놓고 바른미...","[2, 1, 6]",[8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 ...
2,340626904,1104,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연","[[{'index': 0, 'sentence': '[ 홍윤정 기자 ] 8일 서울 올...","[1, 5, 8]",[지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리...
3,340627450,740,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순","[[{'index': 0, 'sentence': '] 박원순 서울시장(사진)이 8일...","[0, 1, 2]",[박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하...
4,340627465,1389,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한","[[{'index': 0, 'sentence': '[ 임근호 기자 ] ""SK(주)와...","[1, 3, 4]",[주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적...


In [16]:
def to_list_str(text) -> List[str]:
    result = []
    for item in text:
        if len(item) < 1: continue
        result.append(item[0]["sentence"])
    return result
    
# 예시
to_list_str(val_news_df_final["text"][100])

['[ 안효주 기자 ] 수천 명의 사람이 요가복을 입고 동시에 천천히 온몸을 구부리는 모습.',
 '이번 요가말라 축제는 롯데백화점이 주최한다.',
 '요가말라 축제에선 요가 전문가들이 고난도의 요가 공연을 선보인다.',
 "롯데백화점의 편집숍 '피트니스 스퀘어'와 '만두카' 등 요가 관련 24개 브랜드가 참여하는 플리마켓도 함께 열린다.",
 '행사를 주최하는 롯데백화점은 라이프스타일 변화에 맞춰 애슬레저 상품군을 대폭 확대하고 있다.']

In [ ]:
val_news_df_final["text"] = val_news_df_final["text"].apply(lambda x: to_list_str(x))
val_news_df_final.head()

In [ ]:
val_news_df_final["text"][3]

In [ ]:
val_news_table = pa.Table.from_pandas(val_news_df_final)

In [ ]:
pq.write_table(val_news_table, '/opt/ml/dataset/Validation/valid.parquet')  # preserve_index=False (omit index)